# Model vulnerability analysis

# scenario discovery

In [ ]:
from ema_workbench.analysis import prim
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ema_workbench import ema_logging, MultiprocessingEvaluator
from ema_workbench import Model, RealParameter, ScalarOutcome, CategoricalParameter, IntegerParameter, BooleanParameter
from ema_workbench.em_framework.samplers import sample_uncertainties
# from ema_workbench.em_framework.evaluators import MC
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation
ema_logging.log_to_stderr(ema_logging.INFO)

# dike_model, planning_steps = get_model_for_problem_formulation(5)

from ema_workbench.util.utilities import load_results

results = load_results('./martijnmc10pol1000scen.tar.gz')

In [ ]:
# do something with the results to make them applicable (maybe)
experiments, outcomes = results

x = experiments
y = outcomes # or all, or some other specific.

In [ ]:
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1)
box1 = prim_alg.find_box()

box1.show_tradeoff()

plt.show

In [ ]:
box1.inspect_tradeoff()

In [ ]:
# select any to inspect (currently 21)
box1.inspect(21)
box1.inspect(21, style='graph')

plt.show()

In [ ]:
box1.select(21)
fig = box1.show_pairs_scatter()
plt.show()

In [ ]:
# drop some variable and reinspect
box1.drop_restriction('Cellulosic cost')
box1.inspect(style='graph')
plt.show()

# subspace partitioning

In [ ]:
from ema_workbench.analysis import feature_scoring
import seaborn as sns


fs = feature_scoring.get_feature_scores_all(experiments, outcomes)
sns.heatmap(fs, cmap='viridis', annot=True)
plt.show()

In [ ]:
from ema_workbench.analysis import dimensional_stacking
dimensional_stacking.create_pivot_plot(x,y, 2, nbins=3)
plt.show()

# sensitivity analysis 

Sobol

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

with SequentialEvaluator(py_model) as evaluator:
    sa_results = evaluator.perform_experiments(scenarios = 50 ,
                                               uncertainty_sampling='sobol')

In [1]:
experiments, outcomes = sa_results
outcomes = {key:outcomes[key][:,0,:] for key in outcomes.keys()}

NameError: name 'sa_results' is not defined

In [ ]:
problem = get_SALib_problem(py_model.uncertainties)

In [ ]:
Si = sobol.analyze(problem, outcomes['prey'][:,-1],
                   calc_second_order=True, print_to_console=False)

In [ ]:
Si_filter = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df = pd.DataFrame(Si_filter, index = problem['names'])
Si_df

extra trees

In [ ]:
extra_trees_results = sa_results_1

tree_experiments, tree_outcomes = extra_trees_results
tree_outcomes = {key:tree_outcomes[key][:,0, -1] for key in tree_outcomes.keys()} # Important for printing, delete redundant array
tree_experiments = tree_experiments.drop(columns=['scenario', 'policy', 'model'], inplace=False)
print(tree_experiments.head(5))

from ema_workbench.analysis import feature_scoring
from ema_workbench import ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)

score = feature_scoring.get_feature_scores_all(tree_experiments, tree_outcomes)
sns.heatmap(score, annot=True)
plt.show()

sobol vs. extra trees

In [ ]:
scores, p_values = feature_scoring.get_rf_feature_scores(tree_experiments,tree_outcomes['prey'],  mode=RuleInductionType.REGRESSION, 
                                                         nr_trees=100, max_features=0.6)
scores

In [2]:
Si_df_1['ST']

NameError: name 'Si_df_1' is not defined